# Human in the Loop

In this section, we will do the following:
- Send external data to running workflows (new user input, updated requirements)
- Update our AI agent to receive feedback and change its execution path without restarting
- Add query handlers to expose Workflow details without interrupting execution

## Setup Notebook

Run the following code blocks to install various packages and tools necessary to run this notebook

**Be sure to add your .env file again. It doesn't persist across notebooks or sesions**

```
LLM_API_KEY = YOUR_API_KEY
LLM_MODEL = openai/gpt-4o
```

In [ ]:
# We'll first install the necessary packages for this workshop.

%pip install --quiet temporalio litellm reportlab python-dotenv

In [ ]:
# Create .env file
with open(".env", "w") as fh:
    fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

    # Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings

import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)
print("LLM API Key", LLM_API_KEY)

In [ ]:
# Mermaid renderer, run at the beginning to setup rendering of diagrams
import base64
from IPython.display import Image, display


def render_mermaid(graph_definition):
    """
    Renders a Mermaid diagram in Google Colab using mermaid.ink.

    Args:
        graph_definition (str): The Mermaid diagram code (e.g., "graph LR; A-->B;").
    """
    graph_bytes = graph_definition.encode("ascii")
    base64_bytes = base64.b64encode(graph_bytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

In [ ]:
# This allows us to run the Temporal Asyncio event loop within the event loop of Jupyter Notebooks
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# Running this will download the Temporal CLI

!curl -sSf https://temporal.download/cli.sh | sh

### Running a Temporal Service

* The Temporal Service brings it all together
* The Temporal Service can be run locally, self-hosted, or you can use Temporal Cloud
* The service acts as the supervisor of your Workflows, Activities, and everything else

In [ ]:
# Start the Temporal Dev Server
import os
import subprocess

command = "/root/.temporalio/bin/temporal server start-dev --ui-port 8000"
temporal_server = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, preexec_fn=os.setsid)

In [ ]:
# Uncomment this to Kill the Temporal Dev Server
# import signal

# os.killpg(os.getpgid(temporal_server.pid), signal.SIGTERM)

## Review the Previous Workflow

Let's quickly review the previous Workflow below to refresh your memory:

### Data Models

* Temporal recommends passing data to and from Activities and Workflows as a single object.
* Use a dataclass for this.

In [ ]:
from dataclasses import dataclass


@dataclass
class LLMCallInput:
    prompt: str
    llm_api_key: str
    llm_model: str


@dataclass
class PDFGenerationInput:
    content: str
    filename: str = "research_pdf.pdf"


@dataclass
class GenerateReportInput:
    prompt: str
    llm_api_key: str
    llm_research_model: str = "openai/gpt-4o"
    llm_image_model: str = "dall-e-3"


@dataclass
class GenerateReportOutput:
    result: str

### Activities

* An Activity is a function/method that is prone to failure and/or non-deterministic.
* Temporal requires all non-deterministic code be run in an Activity
* Activities retry over and over until they succeed or until your customized retry or timeout configuration is hit.
* You define an Activity by adding the `@activity.defn` decorator.

In [ ]:
from temporalio import activity
from litellm import completion, ModelResponse
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch


@activity.defn
def llm_call(input: LLMCallInput) -> ModelResponse:
    response = completion(
        model=input.llm_model, api_key=input.llm_api_key, messages=[{"content": input.prompt, "role": "user"}]
    )
    return response


@activity.defn
def create_pdf(input: PDFGenerationInput) -> str:
    print("Creating PDF document...")

    doc = SimpleDocTemplate(input.filename, pagesize=letter)
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle("CustomTitle", parent=styles["Heading1"], fontSize=24, spaceAfter=30, alignment=1)

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))
    paragraphs = input.content.split("\n\n")
    for para in paragraphs:
        if para.strip():
            p = Paragraph(para.strip(), styles["Normal"])
            story.append(p)
            story.append(Spacer(1, 12))

    doc.build(story)

    print(f"SUCCESS! PDF created: {input.filename}")
    return input.filename

### Workflow

* Activities are orchestrated within a Temporal Workflow.
* Workflows must **not** make API calls, file system calls, or anything non-deterministic. That is what Activities are for.
* Workflows are async, and you define them as a class decorated with the `@workflow.defn` decorator.
* Every Workflow has a **single** entry point, which is an `async` method decorated with `@workflow.run`.

In [ ]:
import asyncio
from datetime import timedelta
import logging

from temporalio import workflow


# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        llm_call_input = LLMCallInput(prompt=input.prompt, llm_api_key=LLM_API_KEY, llm_model=LLM_MODEL)

        research_facts = await workflow.execute_activity(
            llm_call,
            llm_call_input,
            start_to_close_timeout=timedelta(seconds=30),
        )

        workflow.logger.info("Research complete!")

        # Uncomment to add delay
        # await workflow.sleep(timedelta(seconds=20))

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=10),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

### Worker

* Temporal Workflows are run on Workers
* Workers wait for tasks to do, such as executing an Activity or Workflow, and perform them
* Workers find tasks by listenting on a Task Queue
* Workers have Workflows and Activities registered to them so the Worker knows what it is allowed to execute
* This makes the execution of work indirect; _any_ Worker can pick up a registered Workflow or Activity

In [ ]:
from temporalio.client import Client
from temporalio.worker import Worker
import concurrent.futures


async def run_worker() -> None:
    logging.basicConfig(level=logging.INFO)
    logging.getLogger("LiteLLM").setLevel(logging.WARNING)

    client = await Client.connect("localhost:7233", namespace="default")

    # Run the Worker
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as activity_executor:
        worker = Worker(
            client,
            task_queue="research",
            workflows=[GenerateReportWorkflow],
            activities=[llm_call, create_pdf],
            activity_executor=activity_executor,
        )

        print(f"Starting the worker....")
        await worker.run()


## We aren't starting the Worker here, just defining it.

## LLM Powered Decision Making

- Execution path may be determined at runtime by the LLM
- The LLM knows what its **goal** is, current context, available data, what **tools** it has at its disposal, and then determines what step to perform next
- **This is considered _Agentic_**


## Human Interactions in Your AI Applications

 - While some AI applications may operate entirely autonomously, many require human intervention
 - May provide input on launch or at various points throughout the execution
 - Examples:
  - Validation at critical decision points
  - Final review before implementation
  - Feedback loops




### Example: Customer Service Agent with Dynamic Decision Making
The Scenario:
Customer contacts support: "My order hasn't arrived and I need it urgently"

Agentic Workflow in Action:
- LLM decides: Check order status first
- Action result: Order shows "delayed in transit"
- LLM evaluates options and chooses next step based on findings:

- Option A: Process immediate refund (if customer prefers money back)
- Option B: Expedite replacement shipment (if item still needed)
- Option C: Escalate to human agent (if complex issue detected)

Why this Matters
- Cost control
- Security review
- Risk mitigation

In [ ]:
diagram = """
graph LR
    U1[User Interaction optional] -.-> L[LLM makes decisions on what to do next]
    L --> A[Action does what the LLM decided]
    A --> L
    U2[User Interaction optional] -.-> A
"""
render_mermaid(diagram)

## Challenges in Non-Durable Human in the Loop Processes

**Consider the following scenario**

- A user needs to  approve a transaction.
- As they are doing this, the website goes down.
- How do you mitigate this?
  - Do we notify the user to approve the payment again? (creating confusion since the user already 'approved')
  - Do we assume approval and risk processing an unauthorized payment?

Without durable processes, you're forced to choose between security, user experience, and reliability.

## Signals in Temporal

In Temporal, human interaction in Temporal systems is achieved through a [Temporal Signal](https://docs.temporal.io/encyclopedia/workflow-message-passing).

A Signal is a:
* Message sent asynchronously to a running Workflow Execution
* Used to to change the state and control the flow of a Workflow Execution.

*Take 2 minutes to discuss with your neighbor when we might use a Signal, then we'll share answers*



## Example Signal Usage


1. The user initiates a request (Signal).
2. The agents (Activities) determine the next step.
3. Possible agent responses:
    - Ask the user for more information
    - Request permission to run a tool
4. The user confirms the tool run (Signal).
5. The tool runs (API call) and the response is parsed by an LLM and sent back to the user.
6. Steps repeat until the agent reaches its goal.

## Durably Storing Human Interactions

Let's go back to the user approving a payment example now. With Temporal, when the user clicks "approve" in the finance portal, the approval decision gets durably stored.

The user can close the browser, go to lunch, and the Workflow will continue running in the background.

If the payment gateway times out, returns an error or becomes unavailable, Temporal automatically retries the payment step. IT does not need to re-ask the user for approval, because that decision is already durably stored in the Workflow state.

- **No duplicate work** (user does not have to re-approve the same expense)
- **No lost approvals** (Signal persists and processing resumes automatically when system recovesr)
- **No manual intervention** (does not need to manually reconcile failed payments or investigate whetehr an expense was actually approved)
- **Reliable processing** (business can count on approved expenses being paid)

## Developing Signals in Temporal

There are two steps for adding support for a Signal to your Workflow code:

1. **Defining the Signal** - You specify the name and data structure used by Temporal Clients when sending the Signal.
2. **Handling the Signal** - You write code that will be invoked when the Signal is received from a Temporal Client.

## Implementing a Signal

- Create a custom model for the information
- Implement an `__init__` method within your Workflow class and add an instance variable to store the Signal
- Implement a method to handle the Signal, and decorate it with `@workflow.signal`
- Handle the Signal within your Workflow code


## Let's Add a Signal to Our Workflow

We are going to create a Signal to send to our `GenerateReportWorkflow`. After generating the research, before creating a PDF report, the user can review the research. The user has two choices to send to the workflow:
  - Keep the research and generate a PDF report by signaling "keep"
  - Modify the research before generating a PDF report by signaling "edit"

### Create a Model

- Create a model for the Signal to be stored in
- Similar to Activities and Workflows, `dataclasses` are recommended here
- The model can be nested of other classes, such as `StrEnum`

In [ ]:
from enum import StrEnum


class UserDecision(StrEnum):
    KEEP = "KEEP"
    EDIT = "EDIT"
    WAIT = "WAIT"  # UserDecision Signal will start with WAIT as the default state


@dataclass
class UserDecisionSignal:
    decision: UserDecision
    additional_prompt: str = ""

## Defining a Signal Handler

- A Signal is defined in your code and handled in your Workflow Definition.
- A given Workflow Definition can support multiple Signals.
- To define a Signal, set the Signal decorator `@workflow.signal` on the Signal function inside your Workflow class.
- Signal methods define what happens when the signal is received

In [ ]:
import asyncio
from temporalio import workflow


@workflow.defn(sandboxed=False)  # sandboxed=False is a Notebook only requirement. You normally don't do this)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        # Instance variable to store Signal data
        self._user_decision: UserDecisionSignal = UserDecisionSignal(
            decision=UserDecision.WAIT
        )  # UserDecision Signal starts with WAIT as the default state
        self._research_result: str = ""

    # Define the Signal handler
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update instance variable when Signal is received
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )
        # rest of code here

## Waiting for a Signal

- Use `workflow.wait_condition()` to pause until Signal is received (user decides the next step).
- Creates a blocking checkpoint where the workflow stops and waits
- Resumes execution only when the specified condition becomes true

Benefits:
- Prevents resource waste: Workflow doesn't consume compute cycles while waiting. During these waiting periods, the Workflow instance is not consuming CPU or memory. The Worker only “wakes up” when a Workflow needs to perform something.
- Enables true asynchronous interaction: Users can respond at their own pace without timeout

In [ ]:
import asyncio
from temporalio import workflow


@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(
            decision=UserDecision.WAIT
        )  # UserDecision Signal starts with WAIT as the default state
        self._research_result: str = ""

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        continue_agent_loop = True

        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            self._research_result = research_facts["choices"][0]["message"]["content"]

            print(f"Research content: {research_facts}")

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: self._user_decision.decision != UserDecision.WAIT)

            # rest of code here

### Handle the Signal: Storing the Signal state

Once a Signal is received, your Workflow needs to process that data and take appropriate action based on the Signal handler function.

There are three main components:

1. Store Signal State

- Use instance variables to persist signal data across workflow execution
- Can be a simple variable, or a Queue for handling many signals
- Initialize with default values that indicate "no signal received yet"

```
# Instance variable to hold signal data
self._user_decision: UserDecisionSignal = UserDecisionSignal (decision=UserDecision.WAIT) # UserDecision Signal starts with WAIT as the default state
```

### Handle the Signal: Signal Handler Method

- Executes immediately when signal arrives
- In this example, Signal updates workflow state with received data

```
@workflow.signal
async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
    self._user_decision = decision_data
```

## Handle the Signal: React to Signal in Workflow Logic

After receiving a Signal, your Workflow's main execution logic must evaluate the Signal data and determine the appropriate response. This is where the Workflow's business logic intersects with human input.

In this example, we are branching execution based on Signal content.

```
if self._user_decision.decision == UserDecision.KEEP:
    continue_agent_loop = False
elif self._user_decision.decision == UserDecision.EDIT:
    # Modify prompt and reset for next iteration
    self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)
```

- If the Workflow receives `KEEP` as the `UserDecision`, then the Workflow exits the research loop and proceeds to PDF generation.
- If the Workflow receives `EDIT` as the `UserDecision`, then the Workflow incorporates any additional feedback into the prompt, updates the research parameters, and resets the Signal state back to WAIT so it can loop again to regenerate the research and wait for the next user decision.


In [ ]:
# Putting it together, we see we have set the Signal state, created the Signal handler method, and have logic in the Workflow to how it will react to the Signal.

import asyncio
from datetime import timedelta
import logging

from temporalio import workflow


@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
        self._research_result: str = ""

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        continue_agent_loop = True

        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            self._research_result = research_facts["choices"][0]["message"]["content"]

            print(f"Research content: {research_facts}")

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: self._user_decision.decision != UserDecision.WAIT)

            if self._user_decision.decision == UserDecision.KEEP:
                print("User approved the research. Creating PDF...")
                continue_agent_loop = False
            elif self._user_decision.decision == UserDecision.EDIT:
                print("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                    print(f"Regenerating research with updated prompt: {self._current_prompt}")
                else:
                    print("No additional instructions provided. Regenerating with original prompt.")
                llm_call_input.prompt = self._current_prompt

                # Set the decision back to WAIT for the next loop
                self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

## Test the Workflow

Now that your Signal is implemented, you can start the Workflow.

### Run a Worker

As always, code won't execute if a Worker isn't running.

In [ ]:
worker = asyncio.create_task(run_worker())

### Start the Workflow

Now start the Workflow:

In [ ]:
import asyncio

from temporalio.client import Client


client = await Client.connect("localhost:7233", namespace="default")

print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ").strip()

if not prompt:
    prompt = "Give me 5 fun and fascinating facts about tardigrades. Make them interesting and educational!"
    print(f"No prompt entered. Using default: {prompt}")

# Asynchronous start of a Workflow
handle = await client.start_workflow(
    GenerateReportWorkflow.run,
    GenerateReportInput(prompt=prompt, llm_api_key=LLM_API_KEY),
    id="generate-research-report-workflow",
    task_queue="research",
)

print(f"Started workflow. Workflow ID: {handle.id}, RunID {handle.result_run_id}")

## Observing Signals in the Web UI

Now, look at the Web UI. What do you observe?

In [ ]:
# Get the Temporal Web UI URL
from google.colab.output import eval_js

print(eval_js("google.colab.kernel.proxyPort(8000)"))

## Sending Signals

- We can now run the Workflow and send our Signal
- There are multiple ways to send a Signal
  - Using a Temporal Client in an SDK
  - Using the Web UI
  - Using the `temporal` cli
- For this example, we will use a Temporal Client.

## Sending a Signal with the Client

To send a Signal with the Temporal Client, we need to get a "handle" to a specific Workflow Execution, which will be used to interact with that Workflow.

We'll do this with the `get_workflow_handle` method.

```
handle = client.get_workflow_handle(workflow_id)
```

With the handle on the Workflow Execution we want to Signal, we'll then pass in our Signal:

```
signal_data = UserDecisionSignal(decision=UserDecision.KEEP)
await handle.signal("user_decision_signal", signal_data)
```

In [ ]:
# Let's send our Signal from the Client code


async def send_user_decision_signal(client: Client, workflow_id: str) -> None:
    handle = client.get_workflow_handle(
        workflow_id
    )  # Get a handle on the Workflow Execution we want to send a Signal to.

    while True:
        print("\n" + "=" * 80)
        print("1. Type 'keep' to approve the research and create PDF")
        print("2. Type 'edit' to modify the research")
        print("=" * 80)

        decision = input("Your decision (keep/edit): ").strip().lower()

        if decision in {"keep", "1"}:
            signal_data = UserDecisionSignal(decision=UserDecision.KEEP)
            await handle.signal(
                "user_decision_signal", signal_data
            )  # Send our Keep Signal to our Workflow Eexecution we have a handle on
            print("Signal sent to keep research and create PDF")
            break
        if decision in {"edit", "2"}:
            additional_prompt_input = input("Enter additional instructions for the research (optional): ").strip()
            additional_prompt = additional_prompt_input if additional_prompt_input else ""

            signal_data = UserDecisionSignal(decision=UserDecision.EDIT, additional_prompt=additional_prompt)
            await handle.signal(
                "user_decision_signal", signal_data
            )  # Send our Edit Signal to our Workflow Eexecution we have a handle on
            print("Signal sent to regenerate research")

        else:
            print("Please enter either 'keep', 'edit'")

In [ ]:
send_signal = asyncio.run(send_user_decision_signal(client, handle.id))

## Observing Signals in the Web UI

Refresh your Web UI. Look for the `Workflow Execution Signaled` Event. What was the input?

## Queries in Temporal

You can also extract state to show the user with Queries. This can be done during or even after the Workflow Execution. For example, you might want to:

- **Monitor Progress of Long-Running Workflows**: A Client might want to receive updates on the progress, like the percentage of the task completed.
- **Retrieve Results**: Queries can be used to fetch the results of Activities without waiting for the entire Workflow to complete.

## Developing Queries in Temporal

Once a Query is issued, the Client waits for a response from the Workflow. Although Queries are typically used to access the state of an open (running) Workflow Execution, it is also possible to send a Query to a closed Workflow Execution. In either case, there must be at least one running Worker for the Task Queue to which that Workflow belongs.

Your Query should not include any logic that generates commands (such as executing Activities). Remember, Queries are intended to be read-only operations that do not alter the Workflow's state.

## Handling a Query

Let's create a Query which will allow external clients to read the current research content from a running Workflow without interrupting its execution.

Similar to Signals, in the Python SDK, you can handle Queries by annotating a function within your Workflow with `@workflow.query`:

```
@workflow.query
def get_research_result(self) -> str | None:
    """Query to get the current research result"""
    return self._research_result
```

In [ ]:
# Run this to kill the current Worker
x = worker.cancel()

if x:
    print("Worker killed")
else:
    print("Worker was not running. Nothing to kill")

In [ ]:
# Starting the Worker again
worker = asyncio.create_task(run_worker())

# Check if the task is in the set of all tasks
if worker in asyncio.all_tasks():
    # The sleep is necessary because of the async task scheduling in Jupyter
    print("Task is currently active.")  # The Worker now registers the updated Workflow changes
else:
    print("Task is not found in active tasks (might have finished or not yet scheduled).")

In [ ]:
# Adding a Query into our Workflow code

import asyncio
from datetime import timedelta
import logging

from temporalio import workflow


@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
        self._research_result: str = ""

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.query  # Query to get the current research result
    def get_research_result(self) -> str | None:
        return self._research_result

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        continue_agent_loop = True

        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            self._research_result = research_facts["choices"][0]["message"]["content"]

            print(f"Research content: {research_facts}")

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: self._user_decision.decision != UserDecision.WAIT)

            if self._user_decision.decision == UserDecision.KEEP:
                print("User approved the research. Creating PDF...")
                continue_agent_loop = False
            elif self._user_decision.decision == UserDecision.EDIT:
                print("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                    print(f"Regenerating research with updated prompt: {self._current_prompt}")
                else:
                    print("No additional instructions provided. Regenerating with original prompt.")
                llm_call_input.prompt = self._current_prompt

                # Set the decision back to WAIT for the next loop
                self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

## Sending a Query

After defining and setting a handler for the Queries in your Workflow, the next step is to send a Query, which is sent from a Temporal Client. To do this, use the query method. To do this, we will again:

1. Get a handle of the Workflow Execution we will query
2. Send a query with the `query` method.

```
research_result = await handle.query(GenerateReportWorkflow.get_research_result)
```


In [ ]:
# Let's send our Query from the Client code


async def query_research_result(client: Client, workflow_id: str) -> None:
    handle = client.get_workflow_handle(workflow_id)

    try:
        research_result = await handle.query(GenerateReportWorkflow.get_research_result)
        if research_result:
            print(f"Research Result: {research_result}")
        else:
            print("Research Result: Not yet available")

    except Exception as e:
        print(f"Query failed: {e}")

In [ ]:
send_query = asyncio.run(query_research_result(client, handle.id))

## Putting it Together

You can now make a research call, check your results by querying your Workflow, then choose to edit or keep the research!

---
# Exercise 3 - Human in the Loop

* In these exercises you will:
  * **FILL IN**
* Go to the **Exercise** Directory in the Google Drive and open the **Practice** Directory
* Open _01-An-AI-Agent-Practice.ipynb_ and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers
* **You have 5 mins**